In [39]:
import pandas as pd
import numpy as np 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import pickle

In [40]:
data = pd.read_csv("Churn_Modelling.csv")
data.head(3)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1


In [41]:
# Preprocessing data
data.drop(columns=["RowNumber", "CustomerId", "Surname"],axis=1,inplace=True)
data.head(3)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1


In [42]:
# Encoding Categorical Variables
encoder_label_gender = LabelEncoder()
data['Gender'] = encoder_label_gender.fit_transform(data['Gender'])

In [43]:
# One Hot Encoding
encoder_onehot_geography = OneHotEncoder(sparse_output=False)
data['Geography'] = encoder_onehot_geography.fit_transform(data[['Geography']])

In [45]:
# Save the Encoders
with open('labelencoder_gender.pkl','wb') as file:
    pickle.dump(encoder_label_gender, file)

with open('OneHotEncoder_Geography', 'wb') as file:
    pickle.dump(encoder_onehot_geography, file)

In [46]:
# Dividing the dataset into independent and dependent features
X = data.drop(columns=['Exited'], axis=1)
y = data['Exited']

# Split data into training and testing 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [47]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# saving scaler
with open('scaler.pkl', 'wb') as file:
    pickle.dump('scaler', file)

## ANN Implementation

In [50]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

In [51]:
# Build our ANN model
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)), # HL1 connected with input layer
    Dense(32, activation='relu'), # HL2
    Dense(1, activation='sigmoid') # output layer
])

In [53]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                704       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2817 (11.00 KB)
Trainable params: 2817 (11.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [55]:
import tensorflow
opt = tensorflow.keras.optimizers.Adam(learning_rate=0.01)
loss = tensorflow.keras.losses.BinaryCrossentropy()

In [56]:
# compile the model
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

In [57]:
# set up tensor board
log_dir = "logs/fit" +datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [58]:
# set up early stopping
early_stopping_callback = EarlyStopping(monitor='val_loss',patience=5, restore_best_weights=True)

In [59]:
# Training the model
history = model.fit(
    X_train, y_train, validation_data=(X_test, y_test), epochs=100,
    callbacks=[tensorflow_callback, early_stopping_callback]
)

Epoch 1/100


219/219 [==============================] - 3s 4ms/step - loss: 0.4077 - accuracy: 0.8297 - val_loss: 0.3644 - val_accuracy: 0.8500
Epoch 2/100
219/219 [==============================] - 1s 2ms/step - loss: 0.3682 - accuracy: 0.8459 - val_loss: 0.3463 - val_accuracy: 0.8573
Epoch 3/100
219/219 [==============================] - 1s 3ms/step - loss: 0.3570 - accuracy: 0.8511 - val_loss: 0.3443 - val_accuracy: 0.8593
Epoch 4/100
219/219 [==============================] - 1s 2ms/step - loss: 0.3512 - accuracy: 0.8554 - val_loss: 0.3445 - val_accuracy: 0.8580
Epoch 5/100
219/219 [==============================] - 1s 2ms/step - loss: 0.3502 - accuracy: 0.8539 - val_loss: 0.3490 - val_accuracy: 0.8587
Epoch 6/100
219/219 [==============================] - 1s 2ms/step - loss: 0.3467 - accuracy: 0.8566 - val_loss: 0.3435 - val_accuracy: 0.8560
Epoch 7/100
219/219 [==============================] - 1s 2ms/step - loss: 0.3445 - accuracy: 0.8589 - val_loss: 0.3426 - val_accuracy: 0.86